## EDA и предобработка данных

В качестве исходных данных дан анонимизированный датасет пользователей, где для каждого пользователя указан список текущих друзей пользователя. Для каждых двух пользователей известно, как давно они подружились и насколько активно взаимодействуют в соц. сети, обе величины измеряются в некоторых абстрактных величинах. Задача заключается в реализации системы рекомендации друзей в социальной сети, которая для данного пользователя выдает список пользователей, с кем пользователь вероятней всего подружится.

Для начала отсортируем данные по времени для корректной валидации модели. В качестве таргета я решил использовать сумму (`1 + min-max нормализованной intensity взаимодействия пользователей`, где константа является индикатором нахождения этой пары пользователей в друзьях). Такую метрику выбрал для того, чтобы учесть и степень взаимодействия пользователей и сам факт нахождения в друзьях (что может быть более важным для нашего предсказания, ведь существуют ситуации, когда `intensity == 0`). 

Есть несколько дублирующихся пар пользователей с различным уровнем `intensity`, что может объясняться ошибкой в данных, может объясняться несколькими итерациями добавления/удаления из друзей. Для бейзлайна я взял просто усредненное значение метрики в этих ситуациях. 
Также важно заметить, что матрица взаимодействий несимметричная, соответственно для пары юзеров `intensity` - величина несимметричная. Для отстутствующих данных можно транспонировать матрицу и заполнить пропуски в данных. 

```python
    def _read_csv(self) -> pd.DataFrame:
        """
        Read and preprocess friends dataframe
        Returns
        -------
        df : pd.DataFrame
            Dataframe with right columns and custom target
        """
        df = pd.read_csv('data/friends_dataset', header=None)
        df.columns = ['uid1', 'uid2', 'time', 'intensity']
        df = df.groupby(['uid1', 'uid2'])[['time', 'intensity']].mean().reset_index()
        df['time'] = (df['time'] - df['time'].min()) / (df['time'].max() - df['time'].min())
        df['intensity'] = (df['intensity'] - df['intensity'].min()) / (df['intensity'].max() - df['intensity'].min())
        df['target'] = 1 + df['intensity']
        df = df.sort_values(by='time', ascending=False).reset_index(drop=True)
        return df
```

## Baseline model

В качестве эвристической модели я использовал модели поиска ближайших соседей на взвешенном графе друзей. При построении графа стоит обратить внимание на то, что я инвертировал параметр `intensity` (который фактически является весом ребра во взвешенном графе) для удобного примения алгоритмов поиска минимального пути (перевод рёбер в отрицательные числа не работает для Дейкстры, например). 

```python
    def build_graph(self, path) -> nx.Graph:
        """
        Create graph representation of dataset
        Parameters
        ----------
        path : str
            Relative path to csv file
        """
        df = pd.read_csv(path, usecols=['uid1', 'uid2', 'target'])
        df.columns = ['uid1', 'uid2', 'weight']
        df['weight'] = 1 / df['weight']
        g = nx.from_pandas_edgelist(df, source='uid1', target='uid2', edge_attr=['weight'])
        return g
```

###### Реализовал 3 модели:

Пусть list_users - список пользователей, для которых нужно сделать предсказание, G - граф, построенный на тренировочных данных, k - количество предсказаний для каждого юзера. 

    BFS method: 
    - для каждого юзера из list_users в графе G поиском в ширину ищем соседей уровня > 1 и добавляем в список до тех пор, пока не наберём k соседей. 
    - Если после прохождения всего графа (или подграфа до определенного уровня) k соседей не набрано, то дополняем список узлами с наибольшим количеством связей из графа G.

    Dijkstra method: 
    - для каждого юзера из list_users в графе G идём по итератору упорядоченных по удаленности (в терминах веса) вершин и добавляем в список до тех пор, пока не наберём k соседей. 
    - п.2 аналогичен

    Custom method: 
    - для каждого юзера из list_users в графе G подсчитываем количество путей до каждой вершины второго уровня и упорядочиваем по количеству, после чего добавляем в список до тех пор, пока не наберём k соседей. 
    - п.2 аналогичен


```python
    def find_nearest_neighbors(self, user, k, method='BFS') -> list:
        """
        Computes k nearest potential friends for users in list
        Parameters
        ----------
        user : np.array
            User's uids who needs recommendations
        k : int
            The maximum number of predicted elements
        method : int, optional
            Calculation method: Dijkstra and BFS for graphs and custom approach
        Returns
        -------
        recommendations : np.array
            Ordered list with recommended users uids
        """
        if user in self.train_graph.nodes:
            nearest_neighbors = []
            if method == 'BFS':
                for neighbor, level in iter((nx.single_source_shortest_path_length(
                        self.train_graph, user, cutoff=2)).items()):
                    if level > 1:
                        nearest_neighbors.append(neighbor)
                    if len(nearest_neighbors) == k:
                        return nearest_neighbors
            elif method == 'dijkstra':
                for neighbor in iter((nx.single_source_dijkstra(self.train_graph, user, cutoff=2))):
                    if neighbor not in self.neighbors[user]:
                        nearest_neighbors.append(neighbor)
                    if len(nearest_neighbors) == k:
                        return nearest_neighbors
            elif method == 'custom':
                for friend in self.neighbors[user]:
                    for his_friend in self.neighbors[friend]:
                        if his_friend not in self.neighbors[user] and his_friend != user:
                            nearest_neighbors.append(his_friend)
                nearest_neighbors = np.array(Counter(nearest_neighbors).most_common(k))
                if len(nearest_neighbors) > 0:
                    nearest_neighbors = nearest_neighbors[:, 0]
                nearest_neighbors = list(nearest_neighbors[:k])
                if len(nearest_neighbors) > k:
                    return nearest_neighbors
            popular = self.find_popular[:k]
            for i in range(k - len(nearest_neighbors)):
                nearest_neighbors.append(popular[i])
            return nearest_neighbors
        else:
            return self.find_popular[:k]
```

# Проблемы/вопросы/дальнейшие улучшения

Под конец разобрался с интересными фичами на графах (JaccardCoefficent, ResourceAllocation, AdamicAdar), думаю можно сделать достаточно эффективные эвристики с их помощью.
